In [1]:
import scipy.io
import numpy as np
import matplotlib.pyplot as plt
import sys
sys.path.append('../src')  
from data_loading import load_and_combine_mat_data
from preprocessing import preprocess_data
from evaluation import evaluate_model
from feature_selection import get_subject_indices
from sklearn.neural_network import MLPClassifier
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.feature_selection import RFECV
from sklearn.model_selection import train_test_split, StratifiedKFold


In [2]:
# List of file paths
mat_file_paths = [
    '../data/Subject_10.mat', '../data/Subject_9.mat', '../data/Subject_8.mat', '../data/Subject_7.mat', '../data/Subject_6.mat', 
    '../data/Subject_5.mat', '../data/Subject_4.mat', '../data/Subject_3.mat', '../data/Subject_2.mat', '../data/Subject_1.mat'
]

X, y, Channels = load_and_combine_mat_data(mat_file_paths)

Combined EEG Data Shape (Samples, Channels, Trials): (512, 128, 2236)
Combined Labels Shape: (2236,)
Channels: ['A1', 'A2', 'A3', 'A4', 'A5', 'A6', 'A7', 'A8', 'A9', 'A10', 'A11', 'A12', 'A13', 'A14', 'A15', 'A16', 'A17', 'A18', 'A19', 'A20', 'A21', 'A22', 'A23', 'A24', 'A25', 'A26', 'A27', 'A28', 'A29', 'A30', 'A31', 'A32', 'B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B9', 'B10', 'B11', 'B12', 'B13', 'B14', 'B15', 'B16', 'B17', 'B18', 'B19', 'B20', 'B21', 'B22', 'B23', 'B24', 'B25', 'B26', 'B27', 'B28', 'B29', 'B30', 'B31', 'B32', 'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'C10', 'C11', 'C12', 'C13', 'C14', 'C15', 'C16', 'C17', 'C18', 'C19', 'C20', 'C21', 'C22', 'C23', 'C24', 'C25', 'C26', 'C27', 'C28', 'C29', 'C30', 'C31', 'C32', 'D1', 'D2', 'D3', 'D4', 'D5', 'D6', 'D7', 'D8', 'D9', 'D10', 'D11', 'D12', 'D13', 'D14', 'D15', 'D16', 'D17', 'D18', 'D19', 'D20', 'D21', 'D22', 'D23', 'D24', 'D25', 'D26', 'D27', 'D28', 'D29', 'D30', 'D31', 'D32']


In [3]:
# Selected channels for each subject
selected_channels = {
    'Subject 1': ['A29', 'B10', 'B27', 'C5', 'D4', 'D9', 'D24', 'D27'],
    'Subject 2': ['A1', 'A6', 'A12', 'B23', 'B27', 'C29', 'D10', 'D22'],
    'Subject 3': ['A15', 'A20', 'B10', 'B16', 'C3', 'C7', 'D24', 'D32'],
    'Subject 4': ['A14', 'A23', 'B7', 'B11', 'C8', 'C15', 'D6', 'D32'],
    'Subject 5': ['A11', 'A25', 'A32', 'B11', 'B27', 'C18', 'D7', 'D25'],
    'Subject 6': ['A12', 'A20', 'B2', 'B15', 'B24', 'B27', 'C6', 'D32'],
    'Subject 7': ['A14', 'A22', 'A26', 'A29', 'B8', 'B15', 'C9', 'D32'],
    'Subject 8': ['A16', 'A27', 'B6', 'B11', 'C10', 'C31', 'D19', 'D32'],
    'Subject 9': ['A1', 'A20', 'B11', 'B17', 'C8', 'D6', 'D18', 'D23'],
    'Subject 10': ['A3', 'A14', 'A17', 'A27', 'B25', 'C9', 'D23', 'D32']
}

In [4]:
# Combine all channels into a single set to remove duplicates
unique_channels = set(channel for channels in selected_channels.values() for channel in channels)

# Convert to list if needed
unique_channels_list = list(unique_channels)

# Print the unique list of channels
print("Unique channels:", unique_channels_list)
print(len(unique_channels_list))

Unique channels: ['D19', 'D27', 'D4', 'A32', 'A15', 'A26', 'A14', 'A3', 'B10', 'A12', 'D32', 'B25', 'B6', 'B16', 'B8', 'C29', 'A27', 'B23', 'A11', 'C31', 'A17', 'B2', 'D18', 'A25', 'A6', 'A16', 'A29', 'C15', 'A1', 'C5', 'D23', 'D24', 'D6', 'D7', 'C7', 'B27', 'C10', 'C8', 'B17', 'D9', 'C18', 'A23', 'D25', 'A22', 'C3', 'A20', 'B15', 'D10', 'C6', 'B24', 'C9', 'B7', 'B11', 'D22']
54


In [5]:
# Find the indices of the selected channels
selected_indices = [Channels.index(ch) for ch in unique_channels_list]

# Select only the specified channels from the data
X_selected = X[:, selected_indices, :]

In [6]:
X_selected.shape

(512, 54, 2236)

In [7]:
X_preprocessed =  preprocess_data(X_selected)
print(X_preprocessed.shape)

(512, 54, 2236)


# Modeling

## MLP

In [9]:
X = np.transpose(X_preprocessed, (2, 0, 1))  
X = X.reshape(X_preprocessed.shape[2], -1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [10]:
# Define and train the MLP with anti-overfitting measures
mlp = MLPClassifier(
    hidden_layer_sizes=(128, 64),       #  neurons and layers
    activation='relu',                  # ReLU activation function
    solver='adam',                      # Adam optimizer
    alpha=0.01,                         # Increased L2 regularization (weight decay)
    learning_rate_init=0.001,           # Learning rate
    max_iter=300,                       # Set maximum iterations
    random_state=42,
    early_stopping=True,                # Early stopping to prevent overfitting
    validation_fraction=0.2,            # Use 20% of the training data for validation
    n_iter_no_change=10                 # Stop if no improvement for 10 iterations
)

In [11]:
mlp.fit(X_train, y_train)

MLPClassifier(alpha=0.01, early_stopping=True, hidden_layer_sizes=(128, 64),
              max_iter=300, random_state=42, validation_fraction=0.2)

In [12]:
evaluate_model(mlp, X_test, y_test, X_train, y_train)

Test Accuracy: 0.6339285714285714
Classification Report Test:
               precision    recall  f1-score   support

         0.0       0.61      0.62      0.62        97
         1.0       0.63      0.69      0.66       113
         2.0       0.68      0.64      0.66       119
         3.0       0.61      0.59      0.60       119

    accuracy                           0.63       448
   macro avg       0.63      0.63      0.63       448
weighted avg       0.63      0.63      0.63       448

Train Accuracy: 0.930089485458613
Classification Report Train:
               precision    recall  f1-score   support

         0.0       0.95      0.91      0.93       462
         1.0       0.92      0.96      0.94       446
         2.0       0.94      0.92      0.93       440
         3.0       0.92      0.93      0.92       440

    accuracy                           0.93      1788
   macro avg       0.93      0.93      0.93      1788
weighted avg       0.93      0.93      0.93      1788



{'test_accuracy': 0.6339285714285714,
 'test_report': '              precision    recall  f1-score   support\n\n         0.0       0.61      0.62      0.62        97\n         1.0       0.63      0.69      0.66       113\n         2.0       0.68      0.64      0.66       119\n         3.0       0.61      0.59      0.60       119\n\n    accuracy                           0.63       448\n   macro avg       0.63      0.63      0.63       448\nweighted avg       0.63      0.63      0.63       448\n',
 'train_accuracy': 0.930089485458613,
 'train_report': '              precision    recall  f1-score   support\n\n         0.0       0.95      0.91      0.93       462\n         1.0       0.92      0.96      0.94       446\n         2.0       0.94      0.92      0.93       440\n         3.0       0.92      0.93      0.92       440\n\n    accuracy                           0.93      1788\n   macro avg       0.93      0.93      0.93      1788\nweighted avg       0.93      0.93      0.93      1788

### Hyper-parameters tuning

In [13]:
mlp_2 = MLPClassifier(
    hidden_layer_sizes=(128, 32),       # Fewer neurons to reduce complexity
    activation='relu',                 # ReLU activation function
    solver='adam',                     # Adam optimizer
    alpha=0.05,                        # Increased L2 regularization
    learning_rate_init=0.0001,         # Smaller learning rate
    max_iter=200,                      # Reduce max iterations
    random_state=42,
    early_stopping=True,               # Early stopping to prevent overfitting
    validation_fraction=0.2,           # Use 20% of the training data for validation
    n_iter_no_change=5                 # Stop if no improvement for fewer iterations
)


In [14]:
mlp_2.fit(X_train, y_train)

MLPClassifier(alpha=0.05, early_stopping=True, hidden_layer_sizes=(128, 32),
              learning_rate_init=0.0001, n_iter_no_change=5, random_state=42,
              validation_fraction=0.2)

In [15]:
evaluate_model(mlp_2, X_test, y_test, X_train, y_train)

Test Accuracy: 0.7254464285714286
Classification Report Test:
               precision    recall  f1-score   support

         0.0       0.68      0.74      0.71        97
         1.0       0.70      0.71      0.70       113
         2.0       0.71      0.69      0.70       119
         3.0       0.81      0.76      0.78       119

    accuracy                           0.73       448
   macro avg       0.72      0.73      0.72       448
weighted avg       0.73      0.73      0.73       448

Train Accuracy: 0.9451901565995525
Classification Report Train:
               precision    recall  f1-score   support

         0.0       0.95      0.95      0.95       462
         1.0       0.95      0.93      0.94       446
         2.0       0.93      0.95      0.94       440
         3.0       0.95      0.94      0.95       440

    accuracy                           0.95      1788
   macro avg       0.95      0.95      0.95      1788
weighted avg       0.95      0.95      0.95      1788



{'test_accuracy': 0.7254464285714286,
 'test_report': '              precision    recall  f1-score   support\n\n         0.0       0.68      0.74      0.71        97\n         1.0       0.70      0.71      0.70       113\n         2.0       0.71      0.69      0.70       119\n         3.0       0.81      0.76      0.78       119\n\n    accuracy                           0.73       448\n   macro avg       0.72      0.73      0.72       448\nweighted avg       0.73      0.73      0.73       448\n',
 'train_accuracy': 0.9451901565995525,
 'train_report': '              precision    recall  f1-score   support\n\n         0.0       0.95      0.95      0.95       462\n         1.0       0.95      0.93      0.94       446\n         2.0       0.93      0.95      0.94       440\n         3.0       0.95      0.94      0.95       440\n\n    accuracy                           0.95      1788\n   macro avg       0.95      0.95      0.95      1788\nweighted avg       0.95      0.95      0.95      178

In [16]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform
from sklearn.neural_network import MLPClassifier

# Define a narrower parameter distribution to explore
param_dist = {
    'hidden_layer_sizes': [(32,), (64,), (64, 32), (128, 64)],
    'alpha': [0.001, 0.01, 0.05],          # Fewer alpha choices
    'learning_rate_init': [0.001, 0.0001], # Lower learning rates
    'activation': ['relu', 'tanh'],
    'solver': ['adam'],                    # 'adam' is usually more stable; avoid 'sgd'
}

# Set up the RandomizedSearchCV with reduced parameter choices
random_search = RandomizedSearchCV(
    MLPClassifier(max_iter=100, early_stopping=True, random_state=42),
    param_distributions=param_dist,
    n_iter=10,             # Fewer random combinations
    cv=3,                  # 3-fold cross-validation to save resources
    scoring='accuracy',    # Use 'accuracy' or other metrics as needed
    n_jobs=-1              # Use all available cores
)

# Fit the randomized search to a subset of the data if it's too large
try:
    random_search.fit(X_train, y_train)
    print("Best parameters found: ", random_search.best_params_)
    print("Best cross-validation score: ", random_search.best_score_)
except Exception as e:
    print(f"Error: {e}")


Best parameters found:  {'solver': 'adam', 'learning_rate_init': 0.0001, 'hidden_layer_sizes': (128, 64), 'alpha': 0.001, 'activation': 'relu'}
Best cross-validation score:  0.6482102908277405


In [17]:
evaluate_model(random_search, X_test, y_test, X_train, y_train)

Test Accuracy: 0.7165178571428571
Classification Report Test:
               precision    recall  f1-score   support

         0.0       0.66      0.73      0.70        97
         1.0       0.72      0.76      0.74       113
         2.0       0.75      0.70      0.72       119
         3.0       0.72      0.68      0.70       119

    accuracy                           0.72       448
   macro avg       0.72      0.72      0.72       448
weighted avg       0.72      0.72      0.72       448

Train Accuracy: 0.977069351230425
Classification Report Train:
               precision    recall  f1-score   support

         0.0       0.97      0.98      0.98       462
         1.0       0.97      0.98      0.98       446
         2.0       0.98      0.98      0.98       440
         3.0       0.99      0.97      0.98       440

    accuracy                           0.98      1788
   macro avg       0.98      0.98      0.98      1788
weighted avg       0.98      0.98      0.98      1788



{'test_accuracy': 0.7165178571428571,
 'test_report': '              precision    recall  f1-score   support\n\n         0.0       0.66      0.73      0.70        97\n         1.0       0.72      0.76      0.74       113\n         2.0       0.75      0.70      0.72       119\n         3.0       0.72      0.68      0.70       119\n\n    accuracy                           0.72       448\n   macro avg       0.72      0.72      0.72       448\nweighted avg       0.72      0.72      0.72       448\n',
 'train_accuracy': 0.977069351230425,
 'train_report': '              precision    recall  f1-score   support\n\n         0.0       0.97      0.98      0.98       462\n         1.0       0.97      0.98      0.98       446\n         2.0       0.98      0.98      0.98       440\n         3.0       0.99      0.97      0.98       440\n\n    accuracy                           0.98      1788\n   macro avg       0.98      0.98      0.98      1788\nweighted avg       0.98      0.98      0.98      1788

## CNN 
more complex model

In [20]:
X_preprocessed.shape

(512, 54, 2236)

In [22]:
X_cnn = X_preprocessed.transpose(2, 0, 1).reshape(X_preprocessed.shape[2], 512, X_preprocessed.shape[1], 1)
X_train, X_test, y_train, y_test = train_test_split(X_cnn, y, test_size=0.2, random_state=42)

In [25]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, DepthwiseConv2D, SeparableConv2D, AveragePooling2D, Dropout, Flatten, Dense, BatchNormalization, Activation
from tensorflow.keras.optimizers import Nadam

# Define the EEGNet model
def create_eegnet_model(input_shape=(512, 54, 1), num_classes=4):
    model = Sequential()

    # First Conv2D block
    model.add(Conv2D(4, (64, 1), padding='same', input_shape=input_shape, use_bias=False))  # Reduced filters
    model.add(BatchNormalization())

    # Depthwise Conv2D block
    model.add(DepthwiseConv2D((1, 1), use_bias=False, depth_multiplier=2, padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('elu'))
    model.add(AveragePooling2D((1, 4)))  # Pooling along the width
    model.add(Dropout(0.2))

    # Separable Conv2D block
    model.add(SeparableConv2D(8, (1, 16), use_bias=False, padding='same'))  # Reduced filters
    model.add(BatchNormalization())
    model.add(Activation('elu'))

    # Adjust the pooling size here to avoid negative dimensions
    model.add(AveragePooling2D((1, 2)))  # Reduced from (1, 4) to (1, 2)
    model.add(Dropout(0.2))

    # Flatten and classification
    model.add(Flatten())
    model.add(Dense(num_classes, activation='softmax'))

    return model

# Build and compile the model
model = create_eegnet_model()

# Define Nadam optimizer
optimizer = Nadam(learning_rate=0.001)

# Compile the model
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Print model summary
model.summary()


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_1 (Conv2D)               │ (None, 512, 54, 4)     │           256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 512, 54, 4)     │            16 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ depthwise_conv2d_1              │ (None, 512, 54, 8)     │             8 │
│ (DepthwiseConv2D)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 512, 54, 8)     │            32 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_2 (Activation)       │ (None, 512, 54, 8)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ average_pooling2d_2             │ (None, 512, 13, 8)     │             0 │
│ (AveragePooling2D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 512, 13, 8)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ separable_conv2d_1              │ (None, 512, 13, 8)     │           192 │
│ (SeparableConv2D)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_5           │ (None, 512, 13, 8)     │            32 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_3 (Activation)       │ (None, 512, 13, 8)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ average_pooling2d_3             │ (None, 512, 6, 8)      │             0 │
│ (AveragePooling2D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 512, 6, 8)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 24576)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 4)              │        98,308 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 98,844 (386.11 KB)

 Trainable params: 98,804 (385.95 KB)

 Non-trainable params: 40 (160.00 B)

In [26]:
from tensorflow.keras.callbacks import LearningRateScheduler
import numpy as np

# Cyclical learning rate function
def cyclical_lr(step_size, min_lr=1e-5, max_lr=5e-3):
    def clr(epoch):
        cycle = np.floor(1 + epoch / (2 * step_size))
        x = np.abs(epoch / step_size - 2 * cycle + 1)
        lr = min_lr + (max_lr - min_lr) * np.maximum(0, (1 - x))
        return lr
    return clr

# Set cyclical learning rate scheduler
clr_callback = LearningRateScheduler(cyclical_lr(step_size=200))

# Train the model
history = model.fit(X_train, y_train,
                    validation_data=(X_test, y_test),
                    epochs=50,  
                    batch_size=64,  
                    callbacks=[clr_callback])

Epoch 1/50


I0000 00:00:1730334598.210986  197856 service.cc:146] XLA service 0x7d23fc047ba0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1730334598.211063  197856 service.cc:154]   StreamExecutor device (0): NVIDIA A16, Compute Capability 8.6
I0000 00:00:1730334598.211071  197856 service.cc:154]   StreamExecutor device (1): NVIDIA A16, Compute Capability 8.6
2024-10-31 00:29:58.343455: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-10-31 00:29:58.506242: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 8907


 2/28 ━━━━━━━━━━━━━━━━━━━━ 1s 43ms/step - accuracy: 0.2461 - loss: 1.8562 

I0000 00:00:1730334610.409047  197856 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


28/28 ━━━━━━━━━━━━━━━━━━━━ 27s 498ms/step - accuracy: 0.2545 - loss: 1.7251 - val_accuracy: 0.2321 - val_loss: 1.3967 - learning_rate: 1.0000e-05
Epoch 2/50
28/28 ━━━━━━━━━━━━━━━━━━━━ 2s 51ms/step - accuracy: 0.2449 - loss: 1.6800 - val_accuracy: 0.2812 - val_loss: 1.3815 - learning_rate: 3.4950e-05
Epoch 3/50
28/28 ━━━━━━━━━━━━━━━━━━━━ 2s 51ms/step - accuracy: 0.3252 - loss: 1.4502 - val_accuracy: 0.3214 - val_loss: 1.3633 - learning_rate: 5.9900e-05
Epoch 4/50
28/28 ━━━━━━━━━━━━━━━━━━━━ 2s 52ms/step - accuracy: 0.4046 - loss: 1.3011 - val_accuracy: 0.3616 - val_loss: 1.3370 - learning_rate: 8.4850e-05
Epoch 5/50
28/28 ━━━━━━━━━━━━━━━━━━━━ 2s 51ms/step - accuracy: 0.4663 - loss: 1.2181 - val_accuracy: 0.3772 - val_loss: 1.3082 - learning_rate: 1.0980e-04
Epoch 6/50
28/28 ━━━━━━━━━━━━━━━━━━━━ 2s 52ms/step - accuracy: 0.5023 - loss: 1.1660 - val_accuracy: 0.4263 - val_loss: 1.2692 - learning_rate: 1.3475e-04
Epoch 7/50
28/28 ━━━━━━━━━━━━━━━━━━━━ 2s 51ms/step - accuracy: 0.5383 - loss: 1

### hyper-parameters tuning

In [27]:
history_2 = model.fit(X_train, y_train,
                    validation_data=(X_test, y_test),
                    epochs=100,  
                    batch_size=64,  
                    callbacks=[clr_callback])

Epoch 1/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 2s 55ms/step - accuracy: 0.9828 - loss: 0.0452 - val_accuracy: 0.6384 - val_loss: 1.3667 - learning_rate: 1.0000e-05
Epoch 2/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 2s 51ms/step - accuracy: 0.9855 - loss: 0.0447 - val_accuracy: 0.6473 - val_loss: 1.3573 - learning_rate: 3.4950e-05
Epoch 3/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 2s 52ms/step - accuracy: 0.9935 - loss: 0.0341 - val_accuracy: 0.6518 - val_loss: 1.3412 - learning_rate: 5.9900e-05
Epoch 4/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 2s 51ms/step - accuracy: 0.9906 - loss: 0.0354 - val_accuracy: 0.6607 - val_loss: 1.3265 - learning_rate: 8.4850e-05
Epoch 5/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 2s 51ms/step - accuracy: 0.9920 - loss: 0.0266 - val_accuracy: 0.6585 - val_loss: 1.3349 - learning_rate: 1.0980e-04
Epoch 6/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 2s 51ms/step - accuracy: 0.9948 - loss: 0.0210 - val_accuracy: 0.6652 - val_loss: 1.3192 - learning_rate: 1.3475e-04
Epoch 7/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 2s 51ms/step - accuracy: 

In [30]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, DepthwiseConv2D, SeparableConv2D, AveragePooling2D, Dropout, Flatten, Dense, BatchNormalization, Activation
from tensorflow.keras.optimizers import Nadam
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping, LearningRateScheduler
import numpy as np

# Define the EEGNet model with L2 regularization where applicable
def create_eegnet_model(input_shape=(512, 54, 1), num_classes=4):
    model = Sequential()

    # First Conv2D block with L2 regularization
    model.add(Conv2D(3, (64, 1), padding='same', input_shape=input_shape, use_bias=False,
                     kernel_regularizer=l2(0.01)))  # Reduced filters and added L2 regularization
    model.add(BatchNormalization())

    # Depthwise Conv2D block (no kernel_regularizer since not supported)
    model.add(DepthwiseConv2D((1, 1), use_bias=False, depth_multiplier=2, padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('elu'))
    model.add(AveragePooling2D((1, 4)))  # Pooling along the width
    model.add(Dropout(0.3))  # Increased dropout rate

    # Separable Conv2D block with L2 regularization
    model.add(SeparableConv2D(6, (1, 16), use_bias=False, padding='same'))  
    model.add(BatchNormalization())
    model.add(Activation('elu'))
    model.add(AveragePooling2D((1, 2)))  # Reduced pooling size
    model.add(Dropout(0.3))  # Increased dropout rate

    # Flatten and classification
    model.add(Flatten())
    model.add(Dense(num_classes, activation='softmax'))

    return model

# Build and compile the model
model = create_eegnet_model()

# Define Nadam optimizer
optimizer = Nadam(learning_rate=0.001)

# Compile the model
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Print model summary
model.summary()

# Define cyclical learning rate function
def cyclical_lr(step_size, min_lr=1e-5, max_lr=5e-3):
    def clr(epoch):
        cycle = np.floor(1 + epoch / (2 * step_size))
        x = np.abs(epoch / step_size - 2 * cycle + 1)
        lr = min_lr + (max_lr - min_lr) * np.maximum(0, (1 - x))
        return lr
    return clr

# Set cyclical learning rate scheduler
clr_callback = LearningRateScheduler(cyclical_lr(step_size=200))

# Early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Train the model with both cyclical learning rate scheduler and early stopping
history_3 = model.fit(X_train, y_train,
                    validation_data=(X_test, y_test),
                    epochs=50,
                    batch_size=64,
                    callbacks=[clr_callback, early_stopping])  # Added early stopping


Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_4 (Conv2D)               │ (None, 512, 54, 3)     │           192 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_9           │ (None, 512, 54, 3)     │            12 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ depthwise_conv2d_4              │ (None, 512, 54, 6)     │             6 │
│ (DepthwiseConv2D)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_10          │ (None, 512, 54, 6)     │            24 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_5 (Activation)       │ (None, 512, 54, 6)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ average_pooling2d_5             │ (None, 512, 13, 6)     │             0 │
│ (AveragePooling2D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 512, 13, 6)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ separable_conv2d_3              │ (None, 512, 13, 6)     │           132 │
│ (SeparableConv2D)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_11          │ (None, 512, 13, 6)     │            24 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_6 (Activation)       │ (None, 512, 13, 6)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ average_pooling2d_6             │ (None, 512, 6, 6)      │             0 │
│ (AveragePooling2D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ (None, 512, 6, 6)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 18432)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 4)              │        73,732 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 74,122 (289.54 KB)

 Trainable params: 74,092 (289.42 KB)

 Non-trainable params: 30 (120.00 B)

Epoch 1/50


28/28 ━━━━━━━━━━━━━━━━━━━━ 25s 468ms/step - accuracy: 0.2539 - loss: 1.7021 - val_accuracy: 0.2589 - val_loss: 1.4161 - learning_rate: 1.0000e-05
Epoch 2/50
28/28 ━━━━━━━━━━━━━━━━━━━━ 1s 47ms/step - accuracy: 0.2881 - loss: 1.5984 - val_accuracy: 0.2879 - val_loss: 1.4091 - learning_rate: 3.4950e-05
Epoch 3/50
28/28 ━━━━━━━━━━━━━━━━━━━━ 1s 47ms/step - accuracy: 0.2842 - loss: 1.5978 - val_accuracy: 0.3147 - val_loss: 1.3952 - learning_rate: 5.9900e-05
Epoch 4/50
28/28 ━━━━━━━━━━━━━━━━━━━━ 1s 47ms/step - accuracy: 0.3377 - loss: 1.5019 - val_accuracy: 0.3594 - val_loss: 1.3710 - learning_rate: 8.4850e-05
Epoch 5/50
28/28 ━━━━━━━━━━━━━━━━━━━━ 1s 47ms/step - accuracy: 0.3916 - loss: 1.4076 - val_accuracy: 0.3884 - val_loss: 1.3468 - learning_rate: 1.0980e-04
Epoch 6/50
28/28 ━━━━━━━━━━━━━━━━━━━━ 1s 47ms/step - accuracy: 0.4318 - loss: 1.3015 - val_accuracy: 0.4241 - val_loss: 1.3193 - learning_rate: 1.3475e-04
Epoch 7/50
28/28 ━━━━━━━━━━━━━━━━━━━━ 1s 47ms/step - accuracy: 0.4926 - loss: 1